# QLoRA Fine-Tuning for Scent Tag Generation
This notebook fine-tunes a language model on a custom dataset of scent prompts and tag-style responses.

In [ ]:
# Install libraries
!pip install -q --upgrade fsspec==2025.3.2
!pip install -q transformers datasets accelerate peft trl bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.2/376.2 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 836.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Load model and tokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from google.colab import userdata

model_name = 'mistralai/Mistral-7B-Instruct-v0.1'
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype='float16',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4'
)
hf_token = userdata.get('HF_TOKEN')
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, device_map='auto',use_auth_token=hf_token)
tokenizer = AutoTokenizer.from_pretrained(model_name,use_auth_token=hf_token)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:492: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:934: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
# Load and preprocess dataset
from datasets import load_dataset

dataset = load_dataset("json", data_files="scent_dataset_200.jsonl")

def format_prompt(example):
    prompt = f"### Instruction:\n{example['instruction']}\n\n### Response:\n{example['response']}"
    tokenized = tokenizer(prompt, truncation=True, padding='max_length', max_length=512)
    tokenized['labels'] = tokenized['input_ids'].copy()
    return tokenized

tokenized_dataset = dataset['train'].map(format_prompt, remove_columns=dataset['train'].column_names)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
# Apply QLoRA with PEFT
from peft import get_peft_model, LoraConfig, TaskType
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=['q_proj', 'v_proj'],
    task_type=TaskType.CAUSAL_LM,
    lora_dropout=0.05
)
model = get_peft_model(model, peft_config)

training_args = TrainingArguments(
    output_dir="./qlora-scent-model",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    save_total_limit=1,
    save_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

trainer.train()

/tmp/ipython-input-11-3712556249.py:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: fernando-liu-lopez (fernando-liu-lopez-rice-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,2.764500
20,1.450300
30,0.809700
40,0.697400
50,0.650200
60,0.613900
70,0.623400


/usr/local/lib/python3.11/dist-packages/peft/utils/other.py:1221: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-68713011-2e44ec877917469c745ab7ad;93b3d173-333c-4e50-89eb-4a94a8892011)

Cannot access gated repo for url https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1/resolve/main/config.json.
Access to model mistralai/Mistral-7B-Instruct-v0.1 is restricted. You must have access to it and be authenticated to access it. Please log in. - silently ignoring the lookup for the file config.json in mistralai/Mistral-7B-Instruct-v0.1.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:238: UserWarning: Could not find a config file in mistralai/Mistral-7B-Instruct-v0.1 - will assume that the vocabulary was not modified.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/other.py:1221: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request 

TrainOutput(global_step=75, training_loss=1.055129337310791, metrics={'train_runtime': 666.6241, 'train_samples_per_second': 0.9, 'train_steps_per_second': 0.113, 'total_flos': 1.31126500786176e+16, 'train_loss': 1.055129337310791, 'epoch': 3.0})

In [ ]:
import torch

model.eval()

def generate_response(prompt, max_new_tokens=150):
    input_text = f"### Instruction:\n{prompt}\n\n### Response:\n"
    inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            top_p=0.9,
            temperature=0.7,
            repetition_penalty=1.2,
            pad_token_id=tokenizer.eos_token_id
        )

    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    return decoded.split("### Response:")[-1].strip()


In [ ]:
generate_response("What's a good scent for the last day of autumn?")

'ozone, sharp / clean\nambergris, musky / marine\nlavender, floral / calming\ncandle wax, warm / fatty\nwet bark, woody / humid\nrosewater, sweet / romantic\npine smoke, woody / smoky\nold paper, musty / dry\nhoney, sweet / floral\njasmine, floral / ethereal\ncoal smoke, sooty / dense\nmint, cool / crisp\npetrichor, earthy / fresh\nengine oil, metallic / mechanical\nburnt sugar, bitter / sweet\ncedar, woody / grounding\nleather, rich / worn\nsp'

In [ ]:
generate_response("What does it smell like to hug a loved one returning from a long trip?")

'honey, sweet / floral\nmint, cool / crisp\nlavender, floral / calming\nleather, rich / worn\nambergris, musky / marine\nburnt sugar, bitter / sweet\nspiced vanilla, sweet / warming\nengine oil, metallic / mechanical\nozone, sharp / clean\npetrichor, earthy / fresh\ncedar, woody / grounding\nold paper, musty / dry\ncoal smoke, sooty / dense\ncandle wax, warm / fatty\nwet bark, woody / humid\nsulfur, sharp / acrid\nrosewater, sweet / romantic\nsea breeze, sal'

In [ ]:
generate_response("What does it smell like walking into a clean room, with sunlight pouring in from the windows?")

'leather, rich / worn\ncoal smoke, sooty / dense\ncedar, woody / grounding\nrosewater, sweet / romantic\nold paper, musty / dry\nengine oil, metallic / mechanical\njasmine, floral / ethereal\npetrichor, earthy / fresh\ncandle wax, warm / fatty\nwet bark, woody / humid\nspiced vanilla, sweet / warming\nmint, cool / crisp\nambergris, musky / marine\nozone, sharp / clean\nhoney, sweet / floral\nburnt sugar, bitter / sweet\nsea breeze, salty / marine\nlavender,'

In [ ]:
model.save_pretrained("scent-model-final")
tokenizer.save_pretrained("scent-model-final")

/usr/local/lib/python3.11/dist-packages/peft/utils/other.py:1221: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-687132b5-73e821170b0655892e8f8da9;03dbfc6b-cae3-4cff-8c4d-08e7e52a16b5)

Cannot access gated repo for url https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1/resolve/main/config.json.
Access to model mistralai/Mistral-7B-Instruct-v0.1 is restricted. You must have access to it and be authenticated to access it. Please log in. - silently ignoring the lookup for the file config.json in mistralai/Mistral-7B-Instruct-v0.1.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:238: UserWarning: Could not find a config file in mistralai/Mistral-7B-Instruct-v0.1 - will assume that the vocabulary was not modified.
  warnings.warn(


('scent-model-final/tokenizer_config.json',
 'scent-model-final/special_tokens_map.json',
 'scent-model-final/chat_template.jinja',
 'scent-model-final/tokenizer.model',
 'scent-model-final/added_tokens.json',
 'scent-model-final/tokenizer.json')

In [ ]:
# to load model later (not finished)
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("scent-model-final")
tokenizer = AutoTokenizer.from_pretrained("scent-model-final")